### 문제1: 246쪽 문제 6.16

<표 6.10>는 연구대상자들을 임의적으로 대조군과 처리군으로 배정하여 실험한 자료이다. 처리군에 배정된 연구대상자들은 사일리움(psyllium)을 포함하는 시리얼을 연구기간 동안 매일 일정량씩 섭취하였다. 이 연구의 목적은 이 시리얼이 LDL 콜레스테롤을 낮추는 효과가 있는지 알아보는 것이다. 빈도론적 방법이나 베이지안 방법을 사용하여 말기 콜레스테롤 수준에 대한 모형을 구하고자 한다. 초기의 콜레스테롤 수준은 (a) 적절한 점수를 사용하여 공변량(양적 변수)으로 처리하고, (b) 범주형 변수로 처리하여 모형을 적합할 수 있다. 각 경우에 대해서 처리효과를 분석하고 그 결과를 서로 비교하고 해석하라.

<표 6.10> 콜레스테롤 연구에 관한 연습문제 6.16의 자료

||대조군|대조군|대조군|대조군|처리군|처리군|처리군|처리군|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|초기값|≤3.4|3.4~4.1|4.1~4.9|>4.9|≤3.4|3.4~4.1|4.1~4.9|>4.9|
|≤3.4|18|8|0|0|21|4|2|0|
|3.4~4.1|16|30|13|2|17|25|6|0|
|4.1~4.9|0|14|28|7|11|35|36|6|
|>4.9|0|2|15|22|1|5|14|12|


In [15]:
cereal = data.frame(
    init_value = c("very_low","low","high","very_high"),
    con_g1 = as.integer(c(18,16,0,0)),
    con_g2 = as.integer(c(8,30,14,2)),
    con_g3 = as.integer(c(0,13,28,15)),
    con_g4 = as.integer(c(0,2,7,22)),
    treat_g1 = as.integer(c(21,17,11,1)),
    treat_g2 = as.integer(c(4,25,35,5)),
    treat_g3 = as.integer(c(2,6,36,14)),
    treat_g4 = as.integer(c(0,0,6,12))    
)
cereal

init_value,con_g1,con_g2,con_g3,con_g4,treat_g1,treat_g2,treat_g3,treat_g4
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
very_low,18,8,0,0,21,4,2,0
low,16,30,13,2,17,25,6,0
high,0,14,28,7,11,35,36,6
very_high,0,2,15,22,1,5,14,12


In [18]:
# 초기 콜레스테롤 수준을 점수화하여 새로운 열로 추가
cereal$init_score <- as.integer(factor(cereal$init_value, levels = c("very_low", "low", "high", "very_high")))

cereal


init_value,con_g1,con_g2,con_g3,con_g4,treat_g1,treat_g2,treat_g3,treat_g4,init_score
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
very_low,18,8,0,0,21,4,2,0,1
low,16,30,13,2,17,25,6,0,2
high,0,14,28,7,11,35,36,6,3
very_high,0,2,15,22,1,5,14,12,4


In [19]:
# 데이터를 long 형식으로 변환
library(tidyr)

# 대조군 데이터 변환
cereal_long_con <- pivot_longer(cereal, cols = starts_with("con_g"), names_to = "group_con", values_to = "control")
cereal_long_con <- cereal_long_con[, c("init_value", "init_score", "control")]

# 처리군 데이터 변환
cereal_long_treat <- pivot_longer(cereal, cols = starts_with("treat_g"), names_to = "group_treat", values_to = "treatment")
cereal_long_treat <- cereal_long_treat[, c("init_value", "init_score", "treatment")]

# 대조군과 처리군 데이터 결합
cereal_long <- cbind(cereal_long_con, treatment = cereal_long_treat$treatment)

cereal_long

init_value,init_score,control,treatment
<chr>,<int>,<int>,<int>
very_low,1,18,21
very_low,1,8,4
very_low,1,0,2
very_low,1,0,0
low,2,16,17
low,2,30,25
low,2,13,6
low,2,2,0
high,3,0,11


In [ ]:
# 공변량을 포함한 다중 회귀 모형 (초기 콜레스테롤 점수와 그룹에 따른 처리 효과 평가)
# 대조군과 처리군의 효과를 비교하기 위해 lm() 함수를 사용하여 회귀 모델을 적합
model_control <- lm(control ~ init_score, data = cereal_long)
model_treatment <- lm(treatment ~ init_score, data = cereal_long)

# 회귀 분석 결과 확인
summary(model_control)
summary(model_treatment)

In [14]:
# install.packages("VGAM")
library(VGAM)

fit = vglm(cbind(con_g1, con_g2, con_g3, con_g4, treat_g1, treat_g2, treat_g3, treat_g4) ~ init_value, family=cumulative(parallel=TRUE), data=cereal)

summary(fit)

Call:
vglm(formula = cbind(con_g1, con_g2, con_g3, con_g4, treat_g1, 
    treat_g2, treat_g3, treat_g4) ~ init_value, family = cumulative(parallel = TRUE), 
    data = cereal)

Coefficients: 
                    Estimate Std. Error z value Pr(>|z|)    
(Intercept):1       -3.10077    0.23641 -13.116  < 2e-16 ***
(Intercept):2       -1.83903    0.18760  -9.803  < 2e-16 ***
(Intercept):3       -1.05959    0.16994  -6.235 4.52e-10 ***
(Intercept):4       -0.71999    0.16468  -4.372 1.23e-05 ***
(Intercept):5       -0.16018    0.15996  -1.001    0.317    
(Intercept):6        0.84191    0.16838   5.000 5.73e-07 ***
(Intercept):7        2.54817    0.26226   9.716  < 2e-16 ***
init_valuelow        1.31807    0.23389   5.635 1.75e-08 ***
init_valuevery_high -0.02913    0.25740  -0.113    0.910    
init_valuevery_low   1.75238    0.29681   5.904 3.54e-09 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Number of linear predictors:  7 

Names of linear predictors: logitli

In [ ]:
fit0 = vglm(cbind(con_g1, con_g2, con_g3, con_g4, treat_g1, treat_g2, treat_g3, treat_g4) ~ 1, family=cumulative(parallel=TRUE), data=cereal)

lrtest(fit, fit0)

In [12]:
Polviews = read.table("http://www.stat.ufl.edu/~aa/cat/data/Polviews.dat", header=TRUE)
Polviews

fit = vglm(cbind(y1,y2,y3,y4,y5) ~ party + gender, family=cumulative(parallel=TRUE), data=Polviews)
summary(fit)

gender,party,y1,y2,y3,y4,y5
<chr>,<chr>,<int>,<int>,<int>,<int>,<int>
female,dem,25,105,86,28,4
female,repub,0,5,15,83,32
male,dem,20,73,43,20,3
male,repub,0,1,14,72,32


Call:
vglm(formula = cbind(y1, y2, y3, y4, y5) ~ party + gender, family = cumulative(parallel = TRUE), 
    data = Polviews)

Coefficients: 
              Estimate Std. Error z value Pr(>|z|)    
(Intercept):1 -2.12233    0.16875 -12.577   <2e-16 ***
(Intercept):2  0.16892    0.11481   1.471    0.141    
(Intercept):3  1.85716    0.15103  12.297   <2e-16 ***
(Intercept):4  4.65005    0.23496  19.791   <2e-16 ***
partyrepub    -3.63366    0.21785 -16.680   <2e-16 ***
gendermale     0.04731    0.14955   0.316    0.752    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Names of linear predictors: logitlink(P[Y<=1]), logitlink(P[Y<=2]), 
logitlink(P[Y<=3]), logitlink(P[Y<=4])

Residual deviance: 9.8072 on 10 degrees of freedom

Log-likelihood: -35.2032 on 10 degrees of freedom

Number of Fisher scoring iterations: 4 

No Hauck-Donner effect found in any of the estimates


Exponentiated coefficients:
partyrepub gendermale 
0.02641936 1.04844945 

In [21]:
# 데이터 입력
cholesterol_data <- matrix(c(
  18, 8, 0, 0, 21, 4, 2, 0,
  16, 30, 13, 2, 17, 25, 6, 0,
  0, 14, 28, 7, 11, 35, 36, 6,
  0, 2, 15, 22, 1, 5, 14, 12
), nrow = 4, byrow = TRUE)

# 행과 열 이름 설정
rownames(cholesterol_data) <- c("lv1", "lv2", "lv3", "lv4")
colnames(cholesterol_data) <- c("Control_<=3.4", "Control_3.4~4.1", "Control_4.1~4.9", "Control_>4.9",
                                "Treatment_<=3.4", "Treatment_3.4~4.1", "Treatment_4.1~4.9", "Treatment_>4.9")

cholesterol_data
# 카이제곱 검정 수행
chisq_test_result <- chisq.test(cholesterol_data)

# 결과 출력
chisq_test_result


,Control_<=3.4,Control_3.4~4.1,Control_4.1~4.9,Control_>4.9,Treatment_<=3.4,Treatment_3.4~4.1,Treatment_4.1~4.9,Treatment_>4.9
lv1,18,8,0,0,21,4,2,0
lv2,16,30,13,2,17,25,6,0
lv3,0,14,28,7,11,35,36,6
lv4,0,2,15,22,1,5,14,12


Warning message in chisq.test(cholesterol_data):
"Chi-squared approximation may be incorrect"



	Pearson's Chi-squared test

data:  cholesterol_data
X-squared = 249.86, df = 21, p-value < 2.2e-16


In [23]:
# 필요한 라이브러리 로드
library(VGAM)

# 데이터 입력
cholesterol_data <- matrix(c(
  18, 8, 0, 0, 21, 4, 2, 0,
  16, 30, 13, 2, 17, 25, 6, 0,
  0, 14, 28, 7, 11, 35, 36, 6,
  0, 2, 15, 22, 1, 5, 14, 12
), nrow = 4, byrow = TRUE)

# 행과 열 이름 설정
rownames(cholesterol_data) <- c("lv1", "lv2", "lv3", "lv4")
colnames(cholesterol_data) <- c("Control_<=3.4", "Control_3.4~4.1", "Control_4.1~4.9", "Control_>4.9",
                                "Treatment_<=3.4", "Treatment_3.4~4.1", "Treatment_4.1~4.9", "Treatment_>4.9")

# 데이터 프레임으로 변환
cholesterol_df <- as.data.frame(as.table(cholesterol_data))
colnames(cholesterol_df) <- c("Initial_Level", "Group", "Count")

# 'Initial_Level'과 'Group'을 범주형 변수로 변환
cholesterol_df$Initial_Level <- factor(cholesterol_df$Initial_Level, levels = c("lv1", "lv2", "lv3", "lv4"))
cholesterol_df$Group <- factor(cholesterol_df$Group, levels = c("Control_<=3.4", "Control_3.4~4.1", "Control_4.1~4.9", "Control_>4.9",
                                                                "Treatment_<=3.4", "Treatment_3.4~4.1", "Treatment_4.1~4.9", "Treatment_>4.9"))

# Count 값이 정수인지 확인하고, 결측치가 있는지 확인
cholesterol_df$Count <- as.integer(cholesterol_df$Count)  # Count를 정수형으로 변환
cholesterol_df <- cholesterol_df[!is.na(cholesterol_df$Count), ]  # 결측치가 있는 행 제거

# vglm 모델 적합: 다항 로지스틱 회귀 (initial level ~ group)
cholesterol_model <- vglm(Initial_Level ~ Group, family = multinomial(), data = cholesterol_df, weights = Count)

# 모델 결과 출력
summary(cholesterol_model)


Warning message in checkwz(wz, M, trace = trace, wzepsilon = control$wzepsilon):
"18 diagonal elements of the working weights variable 'wz' have been replaced by 1.819e-12"
Warning message in checkwz(wz, M = M, trace = trace, wzepsilon = control$wzepsilon):
"18 diagonal elements of the working weights variable 'wz' have been replaced by 1.819e-12"
Warning message in checkwz(wz, M = M, trace = trace, wzepsilon = control$wzepsilon):
"18 diagonal elements of the working weights variable 'wz' have been replaced by 1.819e-12"
Warning message in checkwz(wz, M = M, trace = trace, wzepsilon = control$wzepsilon):
"18 diagonal elements of the working weights variable 'wz' have been replaced by 1.819e-12"
Warning message in checkwz(wz, M = M, trace = trace, wzepsilon = control$wzepsilon):
"18 diagonal elements of the working weights variable 'wz' have been replaced by 1.819e-12"
Warning message in checkwz(wz, M = M, trace = trace, wzepsilon = control$wzepsilon):
"18 diagonal elements of the worki

ERROR: Error in if (max(abs(ycounts - round(ycounts))) > smallno) warning("converting 'ycounts' to integer in ", : missing value where TRUE/FALSE needed


In [24]:
cholesterol_df

,Initial_Level,Group,Count
,<fct>,<fct>,<int>
1,lv1,Control_<=3.4,18
2,lv2,Control_<=3.4,16
3,lv3,Control_<=3.4,0
4,lv4,Control_<=3.4,0
5,lv1,Control_3.4~4.1,8
6,lv2,Control_3.4~4.1,30
7,lv3,Control_3.4~4.1,14
8,lv4,Control_3.4~4.1,2
9,lv1,Control_4.1~4.9,0


In [27]:
# 필요한 패키지 설치 및 로드
library(VGAM)

# 데이터 입력
data <- data.frame(
  initial_level = factor(c(rep("≤3.4", 4), rep("3.4~4.1", 4), rep("4.1~4.9", 4), rep(">4.9", 4)),
                        levels = c("≤3.4", "3.4~4.1", "4.1~4.9", ">4.9")),
  control_treatment = factor(rep(c("Control", "Treatment"), each = 16), levels = c("Control", "Treatment")),
  count = c(18, 8, 0, 0, 16, 30, 13, 2, 0, 14, 28, 7, 0, 2, 15, 22,
            21, 4, 2, 0, 17, 25, 6, 0, 11, 35, 36, 6, 1, 5, 14, 12)
)
data
# 초기 콜레스테롤 수치에 따른 변화를 분석하기 위한 모델 적합
# vglm 함수 사용: 초기 수준을 종속 변수로, 처리군을 독립 변수로 설정
model <- vglm(count ~ initial_level + control_treatment, family = multinomial(), data = data)

# 결과 출력
summary(model)

# 이 모델은 초기 콜레스테롤 수치와 처리군 간의 상호작용을 분석합니다.


initial_level,control_treatment,count
<fct>,<fct>,<dbl>
≤3.4,Control,18
≤3.4,Control,8
≤3.4,Control,0
≤3.4,Control,0
3.4~4.1,Control,16
3.4~4.1,Control,30
3.4~4.1,Control,13
3.4~4.1,Control,2
4.1~4.9,Control,0


Warning message in checkwz(wz, M = M, trace = trace, wzepsilon = control$wzepsilon):
"12 diagonal elements of the working weights variable 'wz' have been replaced by 1.819e-12"
Warning message in checkwz(wz, M = M, trace = trace, wzepsilon = control$wzepsilon):
"52 diagonal elements of the working weights variable 'wz' have been replaced by 1.819e-12"
Warning message in checkwz(wz, M = M, trace = trace, wzepsilon = control$wzepsilon):
"72 diagonal elements of the working weights variable 'wz' have been replaced by 1.819e-12"
Warning message in checkwz(wz, M = M, trace = trace, wzepsilon = control$wzepsilon):
"104 diagonal elements of the working weights variable 'wz' have been replaced by 1.819e-12"
Warning message in checkwz(wz, M = M, trace = trace, wzepsilon = control$wzepsilon):
"136 diagonal elements of the working weights variable 'wz' have been replaced by 1.819e-12"
Warning message in checkwz(wz, M = M, trace = trace, wzepsilon = control$wzepsilon):
"136 diagonal elements of th

Call:
vglm(formula = count ~ initial_level + control_treatment, family = multinomial(), 
    data = data)

Coefficients: 
                                Estimate Std. Error z value Pr(>|z|)
(Intercept):1                  7.944e+01  3.069e+03      NA       NA
(Intercept):2                  1.615e+01  2.622e+05   0.000    1.000
(Intercept):3                  7.834e+01  3.069e+03   0.026    0.980
(Intercept):4                  6.094e+01  5.842e+03   0.010    0.992
(Intercept):5                  1.615e+01  2.622e+05   0.000    1.000
(Intercept):6                 -1.291e+01  2.243e+05      NA       NA
(Intercept):7                  2.787e+01  1.253e+05   0.000    1.000
(Intercept):8                  7.903e+01  3.069e+03   0.026    0.979
(Intercept):9                 -1.341e+01  2.476e+05      NA       NA
(Intercept):10                 1.615e+01  2.622e+05   0.000    1.000
(Intercept):11                 5.579e+01  6.875e+04   0.001    0.999
(Intercept):12                 2.113e+01  2.275e+0

In [28]:
# 필요한 패키지 설치 및 로드
install.packages("VGAM")
library(VGAM)

# 데이터 준비
# 초기 콜레스테롤 수준을 양적 변수로 변환
initial_level <- c(3.0, 3.75, 4.5, 5.25)  # 각 범위에 해당하는 값
control_treatment <- rep(c("Control", "Treatment"), each = 16)  # 대조군과 처리군

# 각 콜레스테롤 수준별 빈도 수 (주어진 표)
count <- c(18, 8, 0, 0, 16, 30, 13, 2, 0, 14, 28, 7, 0, 2, 15, 22,
           21, 4, 2, 0, 17, 25, 6, 0, 11, 35, 36, 6, 1, 5, 14, 12)

# 데이터를 data.frame으로 구성
data <- expand.grid(initial_level = initial_level, control_treatment = control_treatment)
data$count <- count

# 초기 콜레스테롤 값에 대응하는 최종 콜레스테롤 값 예시 (이 값은 임의로 설정)
# 초기 콜레스테롤 값 + 일정 값 (예: 처리 후 콜레스테롤 수준 변화)
data$final_cholesterol <- data$initial_level + ifelse(data$control_treatment == "Control", 0, -0.5)

# 종속 변수: 최종 콜레스테롤 값에서 초기 콜레스테롤 값을 뺀 차이
data$cholesterol_diff <- data$final_cholesterol - data$initial_level

# vglm 함수를 사용하여 모델 적합
# 'cumulative()'를 사용하여 순위형 로지스틱 회귀 모델 적합
model <- vglm(cholesterol_diff ~ control_treatment, family = cumulative(), data = data)

# 모델 결과 출력
summary(model)


Warning message:
"package 'VGAM' is in use and will not be installed"
Warning message in checkwz(wz, M = M, trace = trace, wzepsilon = control$wzepsilon):
"128 diagonal elements of the working weights variable 'wz' have been replaced by 1.819e-12"
Warning message in checkwz(wz, M = M, trace = trace, wzepsilon = control$wzepsilon):
"128 diagonal elements of the working weights variable 'wz' have been replaced by 1.819e-12"
Warning message in checkwz(wz, M = M, trace = trace, wzepsilon = control$wzepsilon):
"128 diagonal elements of the working weights variable 'wz' have been replaced by 1.819e-12"
Warning message in checkwz(wz, M = M, trace = trace, wzepsilon = control$wzepsilon):
"128 diagonal elements of the working weights variable 'wz' have been replaced by 1.819e-12"
Warning message in vglm.fitter(x = x, y = y, w = w, offset = offset, Xm2 = Xm2, :
"convergence not obtained in 30 IRLS iterations"


Call:
vglm(formula = cholesterol_diff ~ control_treatment, family = cumulative(), 
    data = data)

Coefficients: 
                            Estimate Std. Error z value Pr(>|z|)
(Intercept)                   -28.86   92681.90      NA       NA
control_treatmentTreatment     57.72  131072.00      NA       NA

Name of linear predictor: logitlink(P[Y<=1]) 

Residual deviance: 7.505e-11 on 126 degrees of freedom

Log-likelihood: -3.752e-11 on 126 degrees of freedom

Number of Fisher scoring iterations: 30 

'(Intercept)', 'control_treatmentTreatment'


Exponentiated coefficients:
control_treatmentTreatment 
              1.163195e+25 